In [1]:
from ultralytics import YOLO

try:
    model = YOLO('../models/m60.pt')
    print("Это YOLOv8 модель!")
    print("Классы:", model.names)
except Exception as e:
    print("Не YOLO:", e)


Это YOLOv8 модель!
Классы: {0: 'пора', 1: 'включение', 2: 'подрез', 3: 'прожог', 4: 'трещина', 5: 'наплыв', 6: 'эталон1', 7: 'эталон2', 8: 'эталон3', 9: 'пора-скрытая', 10: 'утяжина', 11: 'несплавление', 12: 'непровар корня'}


In [2]:
model.model

SegmentationModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )


In [3]:
model.names

{0: 'пора',
 1: 'включение',
 2: 'подрез',
 3: 'прожог',
 4: 'трещина',
 5: 'наплыв',
 6: 'эталон1',
 7: 'эталон2',
 8: 'эталон3',
 9: 'пора-скрытая',
 10: 'утяжина',
 11: 'несплавление',
 12: 'непровар корня'}

In [4]:
model.yaml

{'nc': 13,
 'scales': {'n': [0.5, 0.25, 1024],
  's': [0.5, 0.5, 1024],
  'm': [0.5, 1.0, 512],
  'l': [1.0, 1.0, 512],
  'x': [1.0, 1.5, 512]},
 'backbone': [[-1, 1, 'Conv', [64, 3, 2]],
  [-1, 1, 'Conv', [128, 3, 2]],
  [-1, 2, 'C3k2', [256, False, 0.25]],
  [-1, 1, 'Conv', [256, 3, 2]],
  [-1, 2, 'C3k2', [512, False, 0.25]],
  [-1, 1, 'Conv', [512, 3, 2]],
  [-1, 2, 'C3k2', [512, True]],
  [-1, 1, 'Conv', [1024, 3, 2]],
  [-1, 2, 'C3k2', [1024, True]],
  [-1, 1, 'SPPF', [1024, 5]],
  [-1, 2, 'C2PSA', [1024]]],
 'head': [[-1, 1, 'nn.Upsample', ['None', 2, 'nearest']],
  [[-1, 6], 1, 'Concat', [1]],
  [-1, 2, 'C3k2', [512, False]],
  [-1, 1, 'nn.Upsample', ['None', 2, 'nearest']],
  [[-1, 4], 1, 'Concat', [1]],
  [-1, 2, 'C3k2', [256, False]],
  [-1, 1, 'Conv', [256, 3, 2]],
  [[-1, 13], 1, 'Concat', [1]],
  [-1, 2, 'C3k2', [512, False]],
  [-1, 1, 'Conv', [512, 3, 2]],
  [[-1, 10], 1, 'Concat', [1]],
  [-1, 2, 'C3k2', [1024, True]],
  [[16, 19, 22], 1, 'Segment', ['nc', 32, 256]]],
 

In [ ]:
import os
from collections import Counter

dataset_path = "../data/dataset_2"
splits = ["train", "val"]

for split in splits:
    img_folder = os.path.join(dataset_path, split, "images")
    label_folder = os.path.join(dataset_path, split, "labels")

    img_files = [f for f in os.listdir(img_folder) if f.lower().endswith((".jpg", ".png"))]
    label_files = [f for f in os.listdir(label_folder) if f.lower().endswith(".txt")]

    print(f"=== {split.upper()} ===")
    print(f"Images: {len(img_files)}")
    print(f"Labels: {len(label_files)}")
    
    #совпадение имён (без расширения)
    img_names = set(os.path.splitext(f)[0] for f in img_files)
    label_names = set(os.path.splitext(f)[0] for f in label_files)
    print(f"Images without label: {len(img_names - label_names)}")
    print(f"Labels without image: {len(label_names - img_names)}\n")

    class_counter = Counter()
    for lbl_file in label_files:
        with open(os.path.join(label_folder, lbl_file), 'r') as f:
            lines = f.readlines()
            for line in lines:
                cls_id = int(line.strip().split()[0])
                class_counter[cls_id] += 1

    print(f"Статистика по классам в {split}:")
    for cls_id, count in class_counter.items():
        cls_name = model.names.get(cls_id, f"Class_{cls_id}")
        print(f"  {cls_id} ({cls_name}): {count} объектов")
    print("\n")

=== TRAIN ===
Images: 16434
Labels: 16419
Images without label: 149
Labels without image: 134



NameError: name 'Counter' is not defined

Датасет почти совпадает, но есть небольшие расхождения: некоторые изображения без разметки и некоторые разметки без изображений